In [1]:
# Modules
import ipywidgets as widgets
from ipywidgets import interact_manual
import matplotlib.pyplot as plt
import pandas as pd

# needed to use matplotlib with jupyter widgets
%matplotlib notebook

from IPython.display import display, clear_output, Image
import warnings
warnings.filterwarnings("ignore")

from utils import calculate_scores
%matplotlib inline

[2022-02-19 13:28:24,646 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [2]:
# test sample of 200 image
DATA_PATH = 'images_200/'
DF_PATH = 'sm_ar_data.json'
full_df = pd.read_json('results_comparison_200.json')

In [3]:
z=0

In [4]:
full_df.iloc[z:z+1].drop('file_name', axis=1)

,old_hypotheses,old_tok_len,hypotheses,tok_len,old_captions,captions
0,"[رجل, يرتدي, سترة, صفراء]",4,"[امرأ, +, ة, في, ال, +, شارع]",7,رجل يرتدي سترة صفراء,امرأة في الشارع


In [5]:
calculate_scores(full_df.iloc[z], DF_PATH)

,Exp,b1,b2,b3,b4,caption
0,"old processing, old results",9.197,0.0,0.0,0.0,الناس يعبرون الشارع في بلدة صينية مزدحمة.
1,"new processing, new results",27.645,27.645,25.664,21.986,شخص يحمل مظلة صفراء يمشي بالقرب من الرصيف.
2,"new processing, old results",9.444,8.447,6.958,0.0,شخص في الشارع يجر عربة تسوق ويحمل مظلة.
3,"old processing, new results",18.888,18.888,0.0,0.0,امرأة في شارع مزدحم مع عربة تسوق ومظلة.
4,_,_,_,_,_,أشخاص يسيرون وسيارات تسير في شارع بجوار متاجر


In [6]:
row = full_df.iloc[z]

In [7]:
import pandas as pd
from nltk.translate.bleu_score import corpus_bleu
from arabert.preprocess import ArabertPreprocessor
import pyarabic.araby as araby
from utils import print_scores

model_name = "aubmindlab/bert-base-arabertv2"
arabert_prep = ArabertPreprocessor(model_name=model_name)

[2022-02-19 13:28:27,261 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [8]:
# the details of calculate scores
df = pd.read_json(DF_PATH)
ground_truth = df[df['split'] == 'test'].drop(['split', 'tokens', 'tok_len'], axis=1)

# get ground truth 
gt = ground_truth[ground_truth.file_name == row['file_name']].caption.values

# tokenize truth
old_tokens_truth = [araby.tokenize(i) for i in gt]
ara_tokens_truth = [araby.tokenize(arabert_prep.preprocess(i)) for i in gt]

In [29]:
# reverse scores
old_hypo_with_old = row['old_hypotheses']
old_hypo_with_ara = araby.tokenize(arabert_prep.preprocess(row['old_captions']))
ara_hypo_with_old = araby.tokenize(row['captions'])
ara_hypo_with_ara = row['hypotheses']

In [30]:
print("!!!! old model with old preprocessing!!!!")
_,_,_,_ = print_scores([old_tokens_truth], [old_hypo_with_old], prnt=True)
old_tokens_truth, row["old_hypotheses"]

!!!! old model results with old preprocessing!!!!
----- Bleu-n Scores -----
1: 9.197
2: 0.000
3: 0.000
4: 0.000
-------------------------


([['الناس', 'يعبرون', 'الشارع', 'في', 'بلدة', 'صينية', 'مزدحمة', '.'],
  ['شخص', 'يحمل', 'مظلة', 'صفراء', 'يمشي', 'بالقرب', 'من', 'الرصيف', '.'],
  ['شخص', 'في', 'الشارع', 'يجر', 'عربة', 'تسوق', 'ويحمل', 'مظلة', '.'],
  ['امرأة', 'في', 'شارع', 'مزدحم', 'مع', 'عربة', 'تسوق', 'ومظلة', '.'],
  ['أشخاص', 'يسيرون', 'وسيارات', 'تسير', 'في', 'شارع', 'بجوار', 'متاجر']],
 ['رجل', 'يرتدي', 'سترة', 'صفراء'])

In [33]:
print("!!!! old model with arabert preprocessing !!!!")
_,_,_,_ = print_scores([ara_tokens_truth], [old_hypo_with_ara], prnt=True)
ara_tokens_truth, old_hypo_with_ara

!!!! old model results with arabert preprocessing !!!!
----- Bleu-n Scores -----
1: 9.444
2: 8.447
3: 6.958
4: 0.000
-------------------------


([['ال',
   '+',
   'ناس',
   'يعبر',
   '+',
   'ون',
   'ال',
   '+',
   'شارع',
   'في',
   'بلد',
   '+',
   'ة',
   'صيني',
   '+',
   'ة',
   'مزدحم',
   '+',
   'ة',
   '.'],
  ['شخص',
   'يحمل',
   'مظل',
   '+',
   'ة',
   'صفراء',
   'يمشي',
   'ب',
   '+',
   'ال',
   '+',
   'قرب',
   'من',
   'ال',
   '+',
   'رصيف',
   '.'],
  ['شخص',
   'في',
   'ال',
   '+',
   'شارع',
   'يجر',
   'عرب',
   '+',
   'ة',
   'تسوق',
   'و',
   '+',
   'يحمل',
   'مظل',
   '+',
   'ة',
   '.'],
  ['امرأ',
   '+',
   'ة',
   'في',
   'شارع',
   'مزدحم',
   'مع',
   'عرب',
   '+',
   'ة',
   'تسوق',
   'و',
   '+',
   'مظل',
   '+',
   'ة',
   '.'],
  ['أشخاص',
   'يسير',
   '+',
   'ون',
   'و',
   '+',
   'سيار',
   '+',
   'ات',
   'تسير',
   'في',
   'شارع',
   'ب',
   '+',
   'جوار',
   'متاجر']],
 ['رجل', 'يرتدي', 'ستر', '+', 'ة', 'صفراء'])

In [37]:
print("!!!! new model with old preprocessing !!!!")
_,_,_,_ = print_scores([old_tokens_truth], [ara_hypo_with_old], prnt=True)
old_tokens_truth, ara_hypo_with_old

!!!! new model with old preprocessing !!!!
----- Bleu-n Scores -----
1: 18.888
2: 18.888
3: 0.000
4: 0.000
-------------------------


([['الناس', 'يعبرون', 'الشارع', 'في', 'بلدة', 'صينية', 'مزدحمة', '.'],
  ['شخص', 'يحمل', 'مظلة', 'صفراء', 'يمشي', 'بالقرب', 'من', 'الرصيف', '.'],
  ['شخص', 'في', 'الشارع', 'يجر', 'عربة', 'تسوق', 'ويحمل', 'مظلة', '.'],
  ['امرأة', 'في', 'شارع', 'مزدحم', 'مع', 'عربة', 'تسوق', 'ومظلة', '.'],
  ['أشخاص', 'يسيرون', 'وسيارات', 'تسير', 'في', 'شارع', 'بجوار', 'متاجر']],
 ['امرأة', 'في', 'الشارع'])

In [38]:
print("!!!! new model with arabert preprcessing !!!!")
_,_,_,_ = print_scores([ara_tokens_truth], [ara_hypo_with_ara], prnt=True)
ara_tokens_truth, ara_hypo_with_ara

!!!! new model with arabert preprcessing !!!!
----- Bleu-n Scores -----
1: 27.645
2: 27.645
3: 25.664
4: 21.986
-------------------------


([['ال',
   '+',
   'ناس',
   'يعبر',
   '+',
   'ون',
   'ال',
   '+',
   'شارع',
   'في',
   'بلد',
   '+',
   'ة',
   'صيني',
   '+',
   'ة',
   'مزدحم',
   '+',
   'ة',
   '.'],
  ['شخص',
   'يحمل',
   'مظل',
   '+',
   'ة',
   'صفراء',
   'يمشي',
   'ب',
   '+',
   'ال',
   '+',
   'قرب',
   'من',
   'ال',
   '+',
   'رصيف',
   '.'],
  ['شخص',
   'في',
   'ال',
   '+',
   'شارع',
   'يجر',
   'عرب',
   '+',
   'ة',
   'تسوق',
   'و',
   '+',
   'يحمل',
   'مظل',
   '+',
   'ة',
   '.'],
  ['امرأ',
   '+',
   'ة',
   'في',
   'شارع',
   'مزدحم',
   'مع',
   'عرب',
   '+',
   'ة',
   'تسوق',
   'و',
   '+',
   'مظل',
   '+',
   'ة',
   '.'],
  ['أشخاص',
   'يسير',
   '+',
   'ون',
   'و',
   '+',
   'سيار',
   '+',
   'ات',
   'تسير',
   'في',
   'شارع',
   'ب',
   '+',
   'جوار',
   'متاجر']],
 ['امرأ', '+', 'ة', 'في', 'ال', '+', 'شارع'])